In [1]:
import psycopg2
import os

In [11]:
data_dir_path = "../data"

file_table_mapping = (
    ('region.csv','region'),
    ('sales_rep.csv','sales_rep'),
    ('accounts.csv','accounts'),
    ('web_events.csv','web_events'),
    ('orders.csv','orders')
)

In [3]:
dbname_default="postgres"
dbname_created="dwh"
user="username"
password="your pw"
host="localhost"
port="5432"

In [4]:
def connect_db(dbname,user,password,host,port):
    conn = psycopg2.connect(dbname=dbname,
                            user=user,
                            password=password,
                            host=host,
                            port=port)
    return conn

In [8]:
create_tables = [
'''create table REGION(
ID INT,
NAME VARCHAR,
primary key (ID)
);''',
'''
create table SALES_REP(
ID INT,
NAME VARCHAR,
REGION_ID INT,
primary key (ID),
constraint SALESREP_FK 
	foreign key(REGION_ID)
		references region(ID)
);
''',
'''
create table ACCOUNTS (
ID INT,
NAME VARCHAR,
WEBSITE VARCHAR,
LAT FLOAT,
LONG FLOAT,
PRIMARY_POC VARCHAR,
SALES_REP_ID INT,
primary key (ID),
constraint ACCOUNTS_FK
	foreign key(SALES_REP_ID)
		references SALES_REP(ID)
);
''',
'''
create table WEB_EVENTS (
ID INT,
ACCOUNT_ID INT,
OCCURED_AT TIMESTAMP,
CHANNEL VARCHAR,
primary key (ID),
constraint webevents_fk
	foreign key(ACCOUNT_ID)
		references ACCOUNTS(ID)
);
''',
'''
create table ORDERS (
ID INT,
ACCOUNT_ID INT,
OCCURED_AT TIMESTAMP,
STANDARD_QTY INT,
GLOSS_QTY INT,
POSTER_QTY INT,
TOTAL INT,
STANDARD_AMT_USD FLOAT,
GLOSS_AMT_USD FLOAT,
POSTER_AMT_USD FLOAT,
TOTAL_AMT_USD FLOAT,
primary key (ID),
constraint ORDERS_FK
	foreign key(ACCOUNT_ID)
		references ACCOUNTS(ID)
);
'''
]

create_db = '''
create database dwh;
'''

In [6]:
### Connecting to db and database
conn_default = connect_db(dbname=dbname_default,
                            user=user,
                            password=password,
                            port=port,
                            host=host)
conn_default.set_session(autocommit=True)
curr = conn_default.cursor()
curr.execute(create_db)
curr.close()
conn_default.close()

In [14]:
### Connecting to dwh and creating tables
conn = connect_db(dbname=dbname_created,
                            user=user,
                            password=password,
                            port=port,
                            host=host)
conn.set_session(autocommit=True)
curr = conn.cursor()
for table_query in create_tables:
    curr.execute(table_query)
    

In [15]:
### Loading data
for file_name,table_name in file_table_mapping:
        path = os.path.join(data_dir_path,file_name)
        with open(path,'r') as f:
            curr.copy_expert(f"COPY {table_name} FROM STDIN WITH HEADER CSV",f)

In [16]:
curr.close()
conn.close()